# 05. 게이트웨이 (Gateway API)

## 학습 목표

이 노트북에서는 **Kubernetes Gateway API**를 활용한 외부 트래픽 관리를 학습합니다.

**테스트 시나리오:**
1. Gateway 리소스 생성
2. HTTPRoute - 기본 라우팅
3. HTTPRoute - 가중치 기반 라우팅 (Canary)
4. HTTPRoute - 헤더 기반 라우팅

**학습 포인트:**
- Gateway API는 Kubernetes 표준 API (Ingress 대체)
- Istio, Envoy Gateway, Kong 등 다양한 구현체 지원
- VirtualService와 유사하지만 더 표준화된 방식

---

## 사전 조건 체크

In [ ]:
# 환경 확인
!kubectl get pods -n istio-demo | grep -E "httpbin|sleep"

---

## 개념 설명: Gateway API

### Gateway API란?

**Kubernetes Gateway API**는 Ingress의 차세대 대체 API입니다.

| 구분 | Ingress | Gateway API |
|------|---------|-------------|
| 상태 | 레거시 | 차세대 표준 |
| 기능 | 기본 라우팅 | 고급 트래픽 관리 |
| 표준화 | 구현체마다 상이 | 표준화된 스펙 |
| 역할 분리 | 없음 | Gateway(인프라) / HTTPRoute(개발자) |

### 주요 리소스

```
┌─────────────────┐
│  GatewayClass   │  ← 어떤 컨트롤러가 관리할지 (istio, envoy 등)
└────────┬────────┘
         │
┌────────▼────────┐
│    Gateway      │  ← 진입점 정의 (포트, 호스트)
└────────┬────────┘
         │
┌────────▼────────┐
│   HTTPRoute     │  ← 라우팅 규칙 (경로, 헤더, 가중치)
└─────────────────┘
```

---

## 사전 확인: Gateway API CRD

In [ ]:
# Gateway API CRD 설치 확인
result = !kubectl get crd gateways.gateway.networking.k8s.io 2>/dev/null
if result and "gateways.gateway.networking.k8s.io" in str(result):
    print("Gateway API CRD가 설치되어 있습니다.")
else:
    print("Gateway API CRD 설치 중...")
    !kubectl apply -f https://github.com/kubernetes-sigs/gateway-api/releases/download/v1.2.0/standard-install.yaml
    import time
    time.sleep(3)
    print("설치 완료!")

---

## 사전 준비: Backend 서비스

가중치 기반 라우팅 테스트를 위해 Backend v1, v2를 배포합니다.

In [ ]:
# Backend v1, v2 배포 (이전 테스트에서 삭제된 경우)
result = !kubectl get deployment backend-v1 -n istio-demo 2>/dev/null
if "backend-v1" not in str(result):
    print("Backend v1, v2 배포 중...")
    !kubectl apply -f ../01-traffic-management/backend-v1.yaml
    !kubectl apply -f ../01-traffic-management/backend-v2.yaml
    !kubectl apply -f ../05-gateway/backend-v1-service.yaml
    !kubectl apply -f ../05-gateway/backend-v2-service.yaml
    import time
    time.sleep(10)
    print("배포 완료!")
else:
    print("Backend가 이미 배포되어 있습니다.")
    
!kubectl get pods -n istio-demo -l app=backend

---

## 시나리오 1: Gateway 리소스 생성

**Gateway**는 외부 트래픽의 진입점입니다.

```yaml
apiVersion: gateway.networking.k8s.io/v1
kind: Gateway
metadata:
  name: demo-gateway
spec:
  gatewayClassName: istio  # Istio가 이 Gateway 관리
  listeners:
    - name: http
      port: 80
      protocol: HTTP
```

In [ ]:
# Gateway 생성
!kubectl apply -f ../05-gateway/gateway.yaml

In [ ]:
# Gateway 생성 대기
import time
print("Gateway 생성 대기 중 (5초)...")
time.sleep(5)

# Gateway 상태 확인
!kubectl get gateway demo-gateway -n istio-demo

In [ ]:
# Gateway IP 확인
!echo "Gateway IP:"
!kubectl get gateway demo-gateway -n istio-demo -o jsonpath='{.status.addresses[0].value}' 2>/dev/null || echo "(아직 할당되지 않음)"

---

## 시나리오 2: HTTPRoute - 기본 라우팅

**HTTPRoute**는 VirtualService와 유사한 역할을 합니다.

```yaml
apiVersion: gateway.networking.k8s.io/v1
kind: HTTPRoute
metadata:
  name: httpbin-route
spec:
  parentRefs:
    - name: demo-gateway  # 연결할 Gateway
  hostnames:
    - "httpbin.local"
  rules:
    - backendRefs:
        - name: httpbin
          port: 8000
```

In [ ]:
# 기본 HTTPRoute 적용
!kubectl apply -f ../05-gateway/httproute-basic.yaml

In [ ]:
# HTTPRoute 확인
!kubectl get httproute -n istio-demo

In [ ]:
# Gateway를 통한 httpbin 접근 테스트
# (Gateway Pod를 찾아서 테스트하거나, sleep에서 직접 테스트)
print("Gateway를 통한 접근 테스트:")
result = !kubectl get pods -n istio-demo -l istio.io/gateway-name=demo-gateway -o jsonpath='{.items[0].metadata.name}' 2>/dev/null
gateway_pod = result[0] if result else ""

if gateway_pod:
    print(f"Gateway Pod: {gateway_pod}")
    !kubectl exec -n istio-demo {gateway_pod} -- curl -s -H "Host: httpbin.local" http://localhost/get | head -10
else:
    print("Gateway Pod 시작 대기 중... sleep에서 직접 테스트:")
    !kubectl exec -n istio-demo deploy/sleep -c sleep -- curl -s http://httpbin:8000/get | head -5

---

## 시나리오 3: HTTPRoute - 가중치 기반 라우팅 (Canary)

**weight**를 사용하여 트래픽을 분배합니다.

```yaml
rules:
  - backendRefs:
      - name: backend-v1
        port: 8080
        weight: 80  # 80%
      - name: backend-v2
        port: 8080
        weight: 20  # 20%
```

In [ ]:
# Canary HTTPRoute 적용
!kubectl apply -f ../05-gateway/httproute-canary.yaml

In [ ]:
# HTTPRoute 상세 확인
print("=== HTTPRoute Canary 설정 ===")
print()
print("backendRefs:")
print("  - name: backend-v1, weight: 80")
print("  - name: backend-v2, weight: 20")
print()
print("→ 80%는 v1, 20%는 v2로 라우팅")

---

## 시나리오 4: HTTPRoute - 헤더 기반 라우팅

**matches.headers**로 특정 헤더를 매칭하여 라우팅합니다.

```yaml
rules:
  - matches:
      - headers:
          - name: x-version
            value: v2
    backendRefs:
      - name: backend-v2
  - backendRefs:  # 기본값
      - name: backend-v1
```

In [ ]:
# Header-based HTTPRoute 적용
!kubectl apply -f ../05-gateway/httproute-header.yaml

In [ ]:
# HTTPRoute Header 설정 확인
print("=== HTTPRoute Header 기반 라우팅 ===")
print()
print("rules:")
print("  - matches:")
print("      - headers:")
print("          - name: x-version")
print("            value: v2")
print("    backendRefs: [backend-v2]")
print("  - backendRefs: [backend-v1]  # 기본값")
print()
print("→ x-version: v2 헤더 있으면 v2, 없으면 v1")

---

## Gateway API vs Istio API 비교

| 항목 | Istio API | Gateway API |
|------|-----------|-------------|
| **API Group** | networking.istio.io | gateway.networking.k8s.io |
| **진입점 정의** | Gateway | Gateway |
| **라우팅 규칙** | VirtualService | HTTPRoute |
| **표준화** | Istio 전용 | Kubernetes 표준 |
| **호환성** | Istio만 | Istio, Envoy Gateway, Kong 등 |
| **성숙도** | 매우 안정적 | GA (v1.0+) |
| **기능** | 매우 풍부 | 기본 기능 + 확장 |

### 언제 어떤 것을 사용할까?

| 상황 | 권장 |
|------|------|
| Istio 전용 기능 필요 | Istio API |
| 멀티 벤더 지원 필요 | Gateway API |
| 새 프로젝트 시작 | Gateway API |
| 기존 Istio 마이그레이션 | Istio API 유지 또는 점진적 전환 |

---

## 정리 (Cleanup)

In [ ]:
# HTTPRoute 삭제
!kubectl delete -f ../05-gateway/httproute-basic.yaml --ignore-not-found
!kubectl delete -f ../05-gateway/httproute-canary.yaml --ignore-not-found
!kubectl delete -f ../05-gateway/httproute-header.yaml --ignore-not-found

In [ ]:
# Gateway 삭제
!kubectl delete -f ../05-gateway/gateway.yaml --ignore-not-found

In [ ]:
# Backend 리소스 삭제
!kubectl delete -f ../05-gateway/backend-v1-service.yaml --ignore-not-found
!kubectl delete -f ../05-gateway/backend-v2-service.yaml --ignore-not-found
!kubectl delete -f ../01-traffic-management/backend-v1.yaml --ignore-not-found
!kubectl delete -f ../01-traffic-management/backend-v2.yaml --ignore-not-found
print("정리 완료!")

---

## Gateway API 테스트 완료!

### 핵심 정리

| 리소스 | 역할 | 관리 주체 |
|--------|------|----------|
| GatewayClass | 컨트롤러 정의 | 플랫폼 팀 |
| Gateway | 진입점 정의 | 인프라 팀 |
| HTTPRoute | 라우팅 규칙 | 개발 팀 |

**Gateway API 장점:**
- Kubernetes 표준 API
- 역할별 책임 분리
- 다양한 구현체 지원
- 점진적 채택 가능

---

## 모든 테스트 완료!

Istio Service Mesh의 5가지 핵심 모듈을 학습했습니다:

| 모듈 | 노트북 | 핵심 내용 |
|------|--------|----------|
| 환경 검증 | 00-환경검증 | 클러스터, Istio, 샘플 앱 확인 |
| 관측성 | 01-관측성 | Jaeger, Prometheus, 분산 추적 |
| 트래픽 관리 | 02-트래픽관리 | Canary, 헤더 기반 라우팅 |
| 복원력 | 03-복원력 | Timeout, Retry, Circuit Breaker |
| 보안 | 04-보안 | Authorization Policy |
| 게이트웨이 | 05-게이트웨이 | Gateway API |

### 추가 학습 자료

- [Istio 공식 문서](https://istio.io/latest/docs/)
- [Gateway API 공식 문서](https://gateway-api.sigs.k8s.io/)
- [Istio + Gateway API](https://istio.io/latest/docs/tasks/traffic-management/ingress/gateway-api/)